# Data processing for Force Estimation

In [ ]:
import numpy as np
import cv2
import cv2.aruco as aruco
from IPython.display import display, clear_output
import os
import glob
from scipy import interpolate
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
from multiprocessing import Pool
import time

%matplotlib inline

## Plan for video processing:
1. Retrieve images from video.
2. Manually delete frames outside the experiment.

In [ ]:
def video2images(video_file_path, output_path, verbose=False):
    # create directory for output
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    cap = cv2.VideoCapture(video_file_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    resolution = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    n = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            cv2.imwrite(os.path.join(output_path, '{:04d}.png'.format(n)), frame)

            n += 1
            if verbose and n % int(fps) == 0:
                display("Progress: {:.1f}% {}".format(100.0 * n / N, output_path[-14:-8]))
                clear_output(wait=True)

        else:
            break
            
    cap.release()
    
    if verbose:
        print("Retrieved frames forom {}".format(video_file_path))
        print("Stored images in {}".format(output_path))
        print("fps: {}".format(fps))
        print("number of frames: {}".format(N))
        print("resolution: {}".format(resolution))

Get image files from videos (in parallel):

In [ ]:
args = []
for subject_number in range(2, 8):
    for experiment_number in range(1, 11):
        video_file_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/original.MP4'.format(subject_number, experiment_number)
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        args.append((video_file_path, output_path))
        
pool = Pool() # use all available cores
pool.starmap(video2images, args)
pool.close()

#### Bash commands to delete frames outside experiments manually
1. xdg-open 0000.png
2. for i in {0000..0005}; do rm $i.png; done

In [ ]:
# Code for deleting files
subject_number = 0
experiment_number = 0

path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
filenames = glob.glob('{}*.png'.format(path))
filenames.sort()

# find the first/last frames with the LED glowing
first_frame = 0
last_frame = first_frame + 0

# delete frames outside the experiment
frames2delete = filenames[:first_frame] + filenames[last_frame+1:]
for frame in frames2delete:
    os.remove(frame)

#### OR do it automatically:

In [ ]:
def delete_frames_outside_experimets(image_path):
    """
    Delete frames without a glowing LED.
    Assumption: LED lights up after first frame and dies before the last frame.
    """
    filenames = glob.glob('{}*.png'.format(path))
    filenames.sort()
    N = len(filenames)
    
    # get max R-channel LED value for each frame
    max_red = np.zeros(N, dtype=np.int)
    for n in range(N):
        LED = cv2.imread('{}{:04d}.png'.format(image_path, n))[-200:,-200:,2]
        max_red[n] = LED.max()
    
    # detect sudden change in LED brightness
    diff = np.abs(np.diff(max_red))
    threshold = 0.35 * diff.max()
    
    # find the first/last frames with the LED glowing
    first_frame = np.argmax(diff > threshold) + 1
    last_frame = N - 2 - np.argmax(diff[::-1] > threshold)
    
    # delete frames outside the experiment
    frames2delete = filenames[:first_frame] + filenames[last_frame+1:]
    for frame in frames2delete:
        os.remove(frame)

In [ ]:
args = []
for subject_number in range(3, 8):
    for experiment_number in range(1, 11):
        path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        args.append(path)
        
pool = Pool() # use all available cores
pool.map(delete_frames_outside_experimets, args)
pool.close()

In [ ]:
# shift filenames to start with 0
def rename_files(path, verbose=False):
    files = glob.glob('{}/*.png'.format(path))[::-1]
    files.sort()
    
    n = len(files)
    for i in range(n):
        new_name = path + '{:04d}.png'.format(i)
        os.rename(files[i], new_name)
#         if verbose and i % 100 == 0:
#             display("Progress: {:.1f}% {}".format(100.0 * i / n, path[-14:-7]))
#             clear_output(wait=True)

In [ ]:
args = []
for subject_number in range(2, 8):
    for experiment_number in range(1, 11):
        path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        args.append(path)
        
pool = Pool() # use all available cores
pool.map(rename_files, args)
pool.close()

## Decrease the rate twice (for some data that had rate 60, not 30 as all the other)

In [ ]:
def halve_rate(path):
    files = glob.glob('{}/*.png'.format(path))[::-1]
    files.sort()

    files2remove = files[1::2]
    for file in files2remove:
        os.remove(file)
    rename_files(path)

In [ ]:
for subject_number in range(1, 3):
    for experiment_number in range(1, 11):
        path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        halve_rate(path)

## Force data processing:

In [ ]:
class ButterLowpasssFilter:
    def __init__(self, cutoff=3, fs=30.0, order=1):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        self.fs = fs
        self.b, self.a = butter(order, normal_cutoff, btype='low', analog=False)

    def filter(self, data):
        y = filtfilt(self.b, self.a, data, axis=0)
        return y

    def get_t(self, n):
        return n/self.fs

filt = ButterLowpasssFilter()

In [ ]:
def get_force_data(frames_path, force_data_path, frame_rate=30.0, interpolated=True):
    # get frame indexes
    frames = glob.glob('{}*.png'.format(frames_path))[::-1]
#     frame_indexes = np.array([int(x[-8:-4]) for x in frames])
#     frame_indexes.sort()

    # read force measurements
    force_data = np.loadtxt(force_data_path, delimiter=',')
    if not interpolate:
        return force_data[:,0], force_data[:,1], force_data[:,2] # return raw data

    # match frames with the nearest measurements
#     t_frame = frame_indexes * 1000.0 / frame_rate # in milliseconds
    t_frame = np.linspace(0.0, 30000.0, len(frames)) # in milliseconds
    f1 = interpolate.interp1d(force_data[:,0], force_data[:,1], kind='nearest', bounds_error=False, fill_value='extrapolate')
    f2 = interpolate.interp1d(force_data[:,0], force_data[:,2], kind='nearest', bounds_error=False, fill_value='extrapolate')
    vf1 = np.vectorize(f1)
    vf2 = np.vectorize(f2)
    force1_interpolated = vf1(t_frame)
    force2_interpolated = vf2(t_frame)
    
    return t_frame, force1_interpolated, force2_interpolated

In [ ]:
def get_filtered_force_data(frames_path, force_data_path):
    t, f1, f2 = get_force_data(frames_path, force_data_path)
    f1_filtered = filt.filter(f1)
    f2_filtered = filt.filter(f2)
    
    # use raw data for values < 100
    f1_filtered[f1 < 100] = f1[f1 < 100]
    f2_filtered[f2 < 100] = f2[f2 < 100]
    
    return t, f1_filtered, f2_filtered

In [ ]:
subject_number = 6
experiment_number = 1
frames_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
force_data_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/raw_force_data.txt'.format(subject_number, experiment_number)

t_raw, f1_raw, f2_raw = get_force_data(frames_path, force_data_path, interpolated=False)
t, f1_filtered, f2_filtered = get_filtered_force_data(frames_path, force_data_path)

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(t_raw, f1_raw, 'r-', linewidth=1, label='Raw measurements 1')
plt.plot(t, f1_filtered, 'bo', linewidth=2, label='Filtered, synchronized 1')
plt.ylabel('Force sensitive resistor data', fontsize=24)
plt.xlabel('Time [sec]', fontsize=24)
plt.grid()
plt.legend(fontsize=18)
plt.show()

In [ ]:
# save labels in csv format (after filtering)
for subject_number in range(6, 7):
    for experiment_number in range(1, 2):
        frames_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        force_data_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/raw_force_data.txt'.format(subject_number, experiment_number)
        t, f1_filtered, f2_filtered = get_filtered_force_data(frames_path, force_data_path)
        labels = np.concatenate((f1_filtered[..., np.newaxis], f2_filtered[..., np.newaxis]), axis=1)
        
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/'.format(subject_number, experiment_number)
        np.savetxt('{}labels.csv'.format(output_path), labels, delimiter=",")

## Force Derivative

In [ ]:
grad = np.gradient(f1_filtered)

In [ ]:
plt.figure(figsize=(20,15))
# plt.plot(t_raw[1:], dif, 'r-', linewidth=1, label='Raw measurements 1 Diff')
plt.plot(t, grad, 'b-', linewidth=2, label='Filtered, synchronized  1 Diff')
plt.ylabel('Force sensitive resistor data / t', fontsize=24)
plt.xlabel('Time [sec]', fontsize=24)
plt.grid()
plt.ylim(-50, 50)
plt.legend(fontsize=18)
plt.show()

# Visually check if data corresponds

In [ ]:
def play_with_force(image_path, labels_path):
    global labels
    
    labels = np.loadtxt(labels_path, delimiter=',')[:,1:]
    labels = np.mean(labels, axis=1)
    max_label = np.max(labels)

    x1 = 1700
    x2 = x1 + 100
    y2 = 900
    y1 = y2 - 100
    
    N = len(glob.glob('{}*.png'.format(image_path)))
    print(N, labels.shape)
    for n in range(N):
        frame = cv2.imread('{}{:04d}.png'.format(image_path, n))

        k = labels[n] * 255.0 / max_label
        y1 = y2 - int(800 * labels[n] / max_label)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, k), thickness=-1)
        n += 1

        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        
        # Display the resulting frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()

In [ ]:
subject_number = 2
experiment_number = 1
image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames_aligned/'.format(subject_number, experiment_number)
labels_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/labels.csv'.format(subject_number, experiment_number)
play_with_force(image_path, labels_path)

## Aruco Marker

In [ ]:
# interpolate nans in linear numpy array
def interpolated(a):
    nans = np.isnan(a)
    x = np.argwhere(nans).ravel()
    xp = np.argwhere(~nans).ravel()
    fp = a[~nans]
    f = np.interp(x, xp, fp)
    answer = a.copy()
    answer[nans] = f
    return answer

def arg_outliers(coords):
    x = coords[:,0]
    y = coords[:,1]

    sigma_x = np.std(x)
    sigma_y = np.std(y)

    mean_x = np.mean(x)
    mean_y = np.mean(y)

    x_outliers = np.argwhere(np.abs(x - mean_x) > 3 * sigma_x).ravel()
    y_outliers = np.argwhere(np.abs(y - mean_y) > 3 * sigma_y).ravel()

    return np.unique(np.concatenate((x_outliers, y_outliers)))

def interpolated_without_outliers(coords):
    coords_interpolated = coords.copy()

    # delete outliers
    outliers = arg_outliers(coords_interpolated)
    coords_interpolated[outliers, :] = np.nan


    # interpolate
    coords_interpolated[:,0] = interpolated(coords_interpolated[:, 0])
    coords_interpolated[:,1] = interpolated(coords_interpolated[:, 1])

    return coords_interpolated.astype(np.int)

In [ ]:
# detect aruco, save coords
def save_aruco_coords(image_path, output_path, verbose=False):
    N = len(glob.glob('{}*.png'.format(image_path)))
    marker_centers = np.full((N, 2), np.nan)
    for n in range(N):
        
        if verbose and n % 50 == 0:
            display("Progress: {:.1f}% {}".format(100.0 * n / N, image_path[-14:-8]))
            clear_output(wait=True)
        
        # read frame-by-frame
        frame = cv2.imread('{}{:04d}.png'.format(image_path, n))

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_250)
        parameters =  aruco.DetectorParameters_create()

        # detect aruco markers
        corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
        
        count = len(corners)
        if count > 0:
            marker_centers[n] = np.mean(corners[0], axis=1)[0]
            
    np.savetxt('{}aruco_coords.csv'.format(output_path), interpolated_without_outliers(marker_centers), delimiter=",")

In [ ]:
args = []
for subject_number in range(1, 3):
    for experiment_number in range(1, 11):
        image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/'.format(subject_number, experiment_number)
        args.append((image_path, output_path))
        
pool = Pool() # use all available cores
pool.starmap(save_aruco_coords, args)
pool.close()

In [ ]:
# play a video with aruco marker not moving
def play_aligned(image_path, aruco_path, x0=1500, y0=500):
    aruco_coords = np.loadtxt(aruco_path, delimiter=',').astype(np.int)
    N = len(glob.glob('{}*.png'.format(image_path)))
    for n in range(N):
        # read frame-by-frame
        frame = cv2.imread('{}{:04d}.png'.format(image_path, n))

        dx = aruco_coords[n,0] - x0
        dy = aruco_coords[n,1] - y0

        aligned = cv2.copyMakeBorder(frame, max(0, -dy), max(0, dy), max(0, -dx), max(0, dx), cv2.BORDER_REFLECT)[max(0,dy):1080+max(0,dy), max(0,dx):1920+max(0,dx), :]

        aligned = cv2.resize(aligned, (0,0), fx=0.5, fy=0.5)

        cv2.imshow('aligned', aligned)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cv2.destroyAllWindows()

In [ ]:
subject_number = 2
experiment_number = 7
image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
aruco_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/aruco_coords.csv'.format(subject_number, experiment_number)
play_aligned(image_path, aruco_path)

In [ ]:
# mean value of aruco coords
aruco_coords = []
for subject_number in range(1, 8):
    for experiment_number in range(1, 11):
        aruco_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/aruco_coords.csv'.format(subject_number, experiment_number)
        aruco_coords.append(np.loadtxt(aruco_path, delimiter=',').astype(np.int))
np.concatenate(aruco_coords, axis=0).mean(axis=0)

Let's choose (1500, 540) as center coordinates for the aruco marker. 

In [ ]:
x0 = 1500
y0 = 540

In [ ]:
# align frames
def save_aligned_frames(image_path, aruco_path, output_path, x0=1500, y0=500, verbose=False):
    # create directory for output
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    aruco_coords = np.loadtxt(aruco_path, delimiter=',').astype(np.int)
    N = len(glob.glob('{}*.png'.format(image_path)))
    for n in range(N):
        
        if verbose and n % 10 == 0:
            display("Progress: {:.1f}% {}".format(100.0 * n / N, image_path[-14:-8]))
            clear_output(wait=True)
        
        # read frame-by-frame
        frame = cv2.imread('{}{:04d}.png'.format(image_path, n))

        dx = aruco_coords[n,0] - x0
        dy = aruco_coords[n,1] - y0

        aligned = cv2.copyMakeBorder(frame, max(0, -dy), max(0, dy), max(0, -dx), max(0, dx), cv2.BORDER_REFLECT)[max(0,dy):1080+max(0,dy), max(0,dx):1920+max(0,dx), :]

        cv2.imwrite(os.path.join(output_path, '{:04d}.png'.format(n)), aligned)

In [ ]:
args = []
for subject_number in range(1, 3):
    for experiment_number in range(1, 11):
        image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        aruco_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/aruco_coords.csv'.format(subject_number, experiment_number)
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames_aligned/'.format(subject_number, experiment_number)
        args.append((image_path, aruco_path, output_path, x0, y0))
        
pool = Pool() # use all available cores
pool.starmap(save_aligned_frames, args)
pool.close()

## Crop to 224x224

In [ ]:
def save_cropped(image_path, output_path):
    # create directory for output
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    N = len(glob.glob('{}*.png'.format(image_path)))
    for n in range(N):
        # read frame-by-frame
        frame_cropped = cv2.imread('{}{:04d}.png'.format(image_path, n))[540-224:540+224,1500-224-45:1500+224-45,:]
        frame_resized = cv2.resize(frame_cropped, (0,0), fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(output_path, '{:04d}.png'.format(n)), frame_resized)

In [ ]:
args = []
for subject_number in range(1, 8):
    for experiment_number in range(1, 11):
        image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames_aligned/'.format(subject_number, experiment_number)
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames_small/'.format(subject_number, experiment_number)
        args.append((image_path, output_path))
        
pool = Pool() # use all available cores
pool.starmap(save_cropped, args)
pool.close()